In [1]:
from ipywidgets import interact, widgets
interact(lambda x: x**2, x=widgets.IntSlider(min=0, max=10));

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

# 💡 Romer Model of Endogenous Growth

The Romer model marked a significant step in growth theory by providing a framework where **long-run economic growth is endogenous**, driven by intentional activities like research and development (R&D) that create new **ideas (technology, knowledge, $A$)**. This contrasts with the Solow model, where long-run growth relies on exogenous technological progress.

The core insight is that ideas are **non-rival** – once created, they can be used by many people simultaneously without being depleted. This property leads to increasing returns to scale and allows for sustained per capita income growth.

# ⚙️ Key Equations

A simplified version of the Romer model uses the following key equations:

1.  **Idea Production Function:** New ideas ($\dot{A}$ or $\Delta A$) are produced using R&D labor ($L_A$) and the existing stock of ideas ($A$). A common specification leading to constant growth is:
    $$ \dot{A} = \delta_A L_A A $$
    - $\delta_A$: Productivity parameter for R&D workers.
    - $L_A$: Number of workers engaged in R&D.
    - $A$: Existing stock of ideas/knowledge.
    * This implies the *growth rate* of ideas is $g_A = \dot{A}/A = \delta_A L_A$.

2.  **Output Production Function:** Output ($Y$) is produced using ideas ($A$) and labor allocated to producing goods ($L_Y$). A simple form highlighting the role of non-rival ideas is:
    $$ Y = A L_Y $$
    *(Note: This assumes each worker in the goods sector can use the entire stock of ideas $A$.)*

3.  **Labor Allocation:** The total labor force ($L$) grows at rate $n$ and is split between goods production ($L_Y$) and R&D ($L_A$) with a constant fraction $\omega$ (omega) in R&D:
    $$ L_A = \omega L $$
    $$ L_Y = (1 - \omega) L $$
    $$ L_{t+1} = L_t (1 + n) $$

# 📈 Balanced Growth Path (BGP) & Scale Effects

On a Balanced Growth Path (BGP), variables grow at constant rates. In this specific Romer model:

* **Growth Rate of Technology ($g_A$):**
    $$ g_A = \delta_A L_A = \delta_A \omega L $$
    If the labor force $L$ is growing (i.e., $n > 0$), then $L_A$ also grows, and the growth rate of technology *accelerates* over time. This is the strong **scale effect**: a larger population leads to faster technological growth.
* **Growth Rate of Output per Capita ($g_{Y/L}$):**
    Output per capita is $Y/L = A L_Y / L = A (1-\omega)$. The growth rate is:
    $$ g_{Y/L} = g_A $$
    So, output per capita growth also accelerates if $n>0$.

To achieve a *constant* BGP growth rate in this model structure, we often assume **zero population growth ($n=0$)**. In this case:
* $L$ and $L_A$ are constant.
* $g_A = \delta_A \omega L_0$ (constant).
* $g_{Y/L} = g_A$ (constant).
* $g_Y = g_A$ (constant, since $g_L=n=0$).

*(Note: More advanced Romer models modify the idea production function (e.g., $\dot{A} = \delta_A L_A A^{\phi}$ with $\phi<1$) to allow for constant BGP growth even with population growth, mitigating the strong scale effect.)*

The simulation plots levels and growth rates, allowing exploration of these dynamics.

In [2]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Layout
from IPython.display import display, Markdown
import warnings

# Optional: Use a specific style
try:
    plt.style.use('seaborn-v0_8-whitegrid')
except IOError:
    pass # Use default if style not found

def romer_model_simulator(delta_A=0.0005, L0=100.0, n=0.01, omega=0.1, A0=1.0, T=100):
    """
    Simulates a basic Romer model of endogenous growth.
    Y = A * L_Y
    A_dot/A = delta_A * L_A = delta_A * omega * L
    L grows at rate n.

    Args:
        delta_A (float): Productivity of R&D workers (z or delta_A in notes).
        L0 (float): Initial total labor force.
        n (float): Growth rate of the total labor force L.
        omega (float): Fraction of labor force in R&D (0 < omega < 1).
        A0 (float): Initial level of technology/ideas.
        T (int): Time Horizon (number of periods).
    """
    # Ensure T is an integer
    T = int(T)
    if T < 2: T = 2 # Need at least 2 periods for growth calc

    # Input validation
    omega = np.clip(omega, 0.01, 0.99)
    delta_A = max(delta_A, 0)
    n = max(n, 0) # Allow zero population growth
    L0 = max(L0, 1)
    A0 = max(A0, 0.1)

    # Arrays to store results (size T+1 for levels)
    time = np.arange(T + 1)
    A = np.zeros(T + 1)
    L = np.zeros(T + 1)
    L_A = np.zeros(T + 1) # Labor in R&D
    L_Y = np.zeros(T + 1) # Labor in Goods Production
    Y = np.zeros(T + 1) # Total Output
    Y_per_L = np.zeros(T + 1) # Output per worker
    g_A = np.zeros(T)     # Growth rate of A (length T)
    g_L = np.zeros(T)     # Growth rate of L
    g_Y = np.zeros(T)     # Growth rate of Y
    g_Y_per_L = np.zeros(T) # Growth rate of Y/L

    # Initial conditions
    A[0] = A0
    L[0] = L0

    # --- Simulation Loop (t=0 to T-1 to calculate state for t+1) ---
    for t in range(T):
        # Labor allocation for period t
        L_A[t] = omega * L[t]
        L_Y[t] = (1 - omega) * L[t]

        # Production in period t
        Y[t] = A[t] * L_Y[t]
        Y_per_L[t] = A[t] * (1 - omega) # Y/L = A * L_Y / L

        # Calculate next period's state variables
        # Technology growth: A_dot/A = delta_A * L_A => A_t+1 = A_t * (1 + delta_A * L_A_t)
        current_g_A = delta_A * L_A[t]
        A[t+1] = A[t] * (1 + current_g_A)

        # Labor force growth
        L[t+1] = L[t] * (1 + n)

        # Store growth rates (from t to t+1)
        g_A[t] = current_g_A
        g_L[t] = n # Labor growth rate is constant

        # Calculate Y[T] and Y_per_L[T] for the last period
        if t == T-1:
            L_A[T] = omega * L[T]
            L_Y[T] = (1 - omega) * L[T]
            Y[T] = A[T] * L_Y[T]
            Y_per_L[T] = A[T] * (1 - omega)

        # Growth rate of Y and Y/L (handle division by zero)
        if Y[t] > 1e-9: g_Y[t] = (Y[t+1] / Y[t]) - 1
        else: g_Y[t] = 0
        if Y_per_L[t] > 1e-9: g_Y_per_L[t] = (Y_per_L[t+1] / Y_per_L[t]) - 1
        else: g_Y_per_L[t] = 0

    # --- Plotting ---
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.ravel()

    # Plot 0: Levels (Log Scale) - A, L
    axes[0].plot(time, A, label='Technology (A)', color='red', lw=2)
    axes[0].plot(time, L, label=f'Labor (L, n={n:.1%})', color='green', lw=2, linestyle=':')
    axes[0].set_yscale('log')
    axes[0].set_title('Technology & Labor Levels (Log Scale)')
    axes[0].set_xlabel("Time (t)")
    axes[0].set_ylabel("Level (Log Scale)")
    axes[0].legend(fontsize='small')
    axes[0].grid(True, which='both', linestyle='--', alpha=0.6)

    # Plot 1: Output Levels (Log Scale) - Y, Y/L
    axes[1].plot(time, Y, label='Total Output (Y)', color='black', lw=2)
    axes[1].plot(time, Y_per_L, label='Output per Worker (Y/L)', color='purple', lw=2, linestyle='--')
    axes[1].set_yscale('log')
    axes[1].set_title('Output Levels (Log Scale)')
    axes[1].set_xlabel("Time (t)")
    axes[1].set_ylabel("Output (Log Scale)")
    axes[1].legend(fontsize='small')
    axes[1].grid(True, which='both', linestyle='--', alpha=0.6)

    # Plot 2: Growth Rates - gA, g(Y/L), gY, n
    plot_time_g = time[:-1] # Time for growth rates (length T)
    axes[2].plot(plot_time_g, g_A*100, label='Tech. Growth (gA)', color='red', lw=2)
    axes[2].plot(plot_time_g, g_Y_per_L*100, label='Output per Worker Growth (g(Y/L))', color='purple', lw=2, linestyle='--')
    axes[2].plot(plot_time_g, g_Y*100, label='Total Output Growth (gY)', color='black', lw=1.5, alpha=0.7)
    axes[2].axhline(n*100, color='green', linestyle=':', label=f'Labor Growth (n={n:.1%})')
    axes[2].set_title('Growth Rates')
    axes[2].set_xlabel("Time (t)")
    axes[2].set_ylabel("Growth Rate (%)")
    axes[2].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.2f}%'))
    axes[2].legend(fontsize='small')
    axes[2].grid(True, linestyle='--', alpha=0.6)
    # Dynamic ylim for growth rates
    all_growth = np.concatenate([g_A, g_Y_per_L, g_Y, [n]]) * 100
    g_min = min(all_growth.min(), 0)
    g_max = max(all_growth.max(), 0)
    g_range = max(g_max - g_min, 0.1)
    axes[2].set_ylim(g_min - g_range*0.1 - 0.1 , g_max + g_range*0.1 + 0.1)


    # Plot 3: R&D Labor (LA)
    axes[3].plot(time, L_A, label=f'R&D Labor (LA = omega*L)', color='blue', lw=2)
    axes[3].set_title('Labor Allocation to R&D')
    axes[3].set_xlabel("Time (t)")
    axes[3].set_ylabel("Number of R&D Workers (LA)")
    axes[3].legend(fontsize='small')
    axes[3].grid(True, linestyle='--', alpha=0.6)
    axes[3].set_ylim(bottom=0)


    fig.suptitle(f"Romer Endogenous Growth Model (gA = δA*ω*L)", fontsize=16, y=1.03)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

    # --- Display Final Growth Rates & BGP Info ---
    final_gA = g_A[-1] if T > 0 else np.nan
    final_gYperL = g_Y_per_L[-1] if T > 0 else np.nan
    final_gY = g_Y[-1] if T > 0 else np.nan

    # BGP growth rate depends on whether L is growing (scale effect)
    bgp_gA_explanation = f"δA * ω * L({T}) = {delta_A:.4f} * {omega:.2f} * {L[-1]:.1f} ≈ {final_gA:.3%}" if T>0 else "N/A"
    bgp_gYperL_explanation = f"gA ≈ {final_gA:.3%}" if T>0 else "N/A"
    bgp_gY_explanation = f"gA + n ≈ {final_gA + n:.3%}" if T>0 else "N/A"


    results_md = f"""
    ### 📈 Growth Results (Approximate Final Rates at t={T-1}):

    * **Technology Growth (gA):** {final_gA:.3%}
    * **Output per Worker Growth (g(Y/L)):** {final_gYperL:.3%}
    * **Total Output Growth (gY):** {final_gY:.3%}
    * **Labor Growth (n):** {n:.3%}

    **Balanced Growth Path (BGP) Predictions:**
    * $g_A = \\delta_A \\omega L_t$. If $n>0$, $L_t$ grows, causing $g_A$ to accelerate (Scale Effect). If $n=0$, $g_A$ is constant.
    * $g_{{Y/L}} = g_A$.
    * $g_Y = g_A + n$.

    * **Final Calculated gA:** {bgp_gA_explanation}
    * **Final Calculated g(Y/L):** {bgp_gYperL_explanation}
    * **Final Calculated gY:** {bgp_gY_explanation}
    """
    display(Markdown(results_md))


# --- Create Interactive Widgets ---
style = {'description_width': 'initial'}
layout = Layout(width='95%')

interact(
    romer_model_simulator,
    delta_A=FloatSlider(value=0.0005, min=0.0, max=0.005, step=0.0001, description='R&D Productivity (delta_A δA):', style=style, layout=layout, readout_format='.4f'),
    L0=FloatSlider(value=100.0, min=10, max=1000, step=10, description='Initial Labor (L0):', style=style, layout=layout, readout_format='.0f'),
    n=FloatSlider(value=0.01, min=0.0, max=0.05, step=0.002, description='Labor Growth (n):', style=style, layout=layout, readout_format='.1%'),
    omega=FloatSlider(value=0.1, min=0.01, max=0.5, step=0.01, description='R&D Share (omega ω):', style=style, layout=layout, readout_format='.2f'),
    A0=FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description='Initial Tech (A0):', style=style, layout=layout, readout_format='.1f'),
    T=IntSlider(value=100, min=10, max=200, step=5, description='Time Periods (T):', style=style, layout=layout)
);


interactive(children=(FloatSlider(value=0.0005, description='R&D Productivity (delta_A δA):', layout=Layout(wi…

# 📈 The Romer Growth Model
# Ideas, Innovation, and Endogenous Growth

In the Romer model, **economic growth** is driven by the creation of new ideas.  
Unlike the Solow model, where long-run growth is exogenous, Romer shows how human capital and R&D efforts endogenously determine growth rates.

> *"Ideas are different. They are nonrival and can be used by many people at the same time."* – Paul Romer

This simulation shows how parameters like the productivity of researchers, the fraction of labor allocated to R&D, and the size of the workforce drive **idea growth** and hence **output per person**.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
import matplotlib.ticker as mtick

plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['figure.figsize'] = (10, 6)

In [4]:
def romer_growth_model(L=100, l=0.2, phi=0.5, A0=1, z=0.05, T=50):
    A = [A0]
    Y = []

    for t in range(T):
        A_next = A[-1] * (1 + z * phi * l * L)
        A.append(A_next)
        Y.append((1 - l) * L * A[-1])  # Output = non-R&D labor × idea stock

    t_vals = list(range(T))

    fig, ax = plt.subplots()
    ax.plot(t_vals, Y, label='Output per Person', color="mediumblue")
    ax.plot(t_vals, A[:-1], '--', label='Ideas (A)', color="gray")
    ax.set_title('Romer Model: Endogenous Growth Over Time')
    ax.set_xlabel('Time (Years)')
    ax.set_ylabel('Level')
    ax.legend()
    ax.grid(True)
    ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.0f}'))
    plt.tight_layout()
    plt.show()

In [5]:
interact(
    romer_growth_model,
    L=IntSlider(value=100, min=50, max=200, step=10, description='Labor (L)'),
    l=FloatSlider(value=0.2, min=0.05, max=0.5, step=0.01, description='R&D share (l)'),
    phi=FloatSlider(value=0.5, min=0.1, max=1.0, step=0.05, description='Productivity (φ)'),
    A0=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='Initial A'),
    z=FloatSlider(value=0.05, min=0.01, max=0.1, step=0.005, description='Scaling z'),
    T=IntSlider(value=50, min=20, max=100, step=5, description='Time Horizon'),
)

interactive(children=(IntSlider(value=100, description='Labor (L)', max=200, min=50, step=10), FloatSlider(val…

<function __main__.romer_growth_model(L=100, l=0.2, phi=0.5, A0=1, z=0.05, T=50)>

# 📌 Core Equation

The production of new ideas follows:

$$
\Delta A = z \\cdot \phi \\cdot l \\cdot L \\cdot A
$$

This exponential dynamic implies that with **more people working in R&D**, **higher productivity**, or **larger initial knowledge**, growth accelerates.


This model shows why **population size** and **research incentives** matter enormously for long-run prosperity.

*Explore how different parameters change the pace of innovation and output per person.*